In [1]:
import os
from glob import glob
import pandas as pd
import re
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.spatial.distance import minkowski,chebyshev
plt.style.use('ggplot')

In [2]:
def extract_distance_features(df):
    """Returns a DataFrame with columns as the Distances and some geometric measures between two or more Joints."""

    SpineBase = df.iloc[:,0:3].values       #0
    SpineMid = df.iloc[:,3:6].values        #1
    Neck = df.iloc[:,6:9].values            #2
    Head = df.iloc[:,9:12].values           #3
    ShoulderLeft = df.iloc[:,12:15].values  #4
    ElbowLeft = df.iloc[:,15:18].values     #5
    WristLeft = df.iloc[:,18:21].values     #6
    HandLeft = df.iloc[:,21:24].values      #7
    ShoulderRight = df.iloc[:,24:27].values #8
    ElbowRight = df.iloc[:,27:30].values    #9
    WristRight = df.iloc[:,30:33].values    #10
    HandRight = df.iloc[:,33:36].values     #11
    HipLeft = df.iloc[:,36:39].values       #12
    KneeLeft = df.iloc[:,39:42].values      #13
    AnkleLeft = df.iloc[:,42:45].values     #14
    FootLeft = df.iloc[:,45:48].values      #15
    HipRight = df.iloc[:,48:51].values      #16
    KneeRight = df.iloc[:,51:54].values     #17
    AnkleRight = df.iloc[:,54:57].values    #18
    FootRight = df.iloc[:,57:60].values     #19
    SpineShoulder = df.iloc[:,60:63].values #20
    HandTipLeft = df.iloc[:,63:66].values   #21
    ThumbLeft = df.iloc[:,66:69].values     #22
    HandTipRight = df.iloc[:,69:72].values  #23
    ThumbRight = df.iloc[:,72:75].values    #24
    
    points = [SpineBase,SpineMid,Neck,Head,ShoulderLeft,ElbowLeft,WristLeft,HandLeft,ShoulderRight,ElbowRight,WristRight,HandRight,HipLeft,KneeLeft,AnkleLeft,FootLeft,HipRight,KneeRight,AnkleRight,FootRight,SpineShoulder,HandTipLeft,ThumbLeft,HandTipRight,ThumbRight]

    def calculate_distance(v1,v2):
        dist = []
        for i in range(df.shape[0]):
            dist.append(np.sqrt(np.sum(np.square(v1[i] - v2[i])))) ## Euclidean Distance
            # dist.append(np.sum(abs(v1[i]-v2[i]))) ## Manhattan Distance
            # dist.append(minkowski(v1[i],v2[i],p=3)) ## Minkowski Distance
            # dist.append(chebyshev(v1[i],v2[i])) ## Chebyshev Distance
        return dist

    def triangle_area(a,b,c):
        v1 = b-a
        v2 = c-a
        area = []
        for i in range(df.shape[0]):
            area.append(1/2 * (np.linalg.norm(np.cross(v1[i],v2[i]))))
        return area

    def calculate_centroid(a,b,c):
        x_centroid = []
        y_centroid = []
        z_centroid=[]
        for i in range(df.shape[0]):
            x_centroid.append((a[i][0] + b[i][0] + c[i][0]) / 3)
            y_centroid.append((a[i][1] + b[i][1] + c[i][1]) / 3)
            z_centroid.append((a[i][2] + b[i][2] + c[i][2]) / 3)
        return x_centroid, y_centroid,z_centroid

    column = {k:0 for k in range(len(points))}
    print(column)
    
    for i in range(len(points)):
        column[i] = calculate_distance(points[i], points[1])

    column[25] = calculate_distance(points[13],points[17])   ## KneeLeft , KneeRight --> Imp
    column[26] = calculate_distance(points[15],points[19])   ## FootLeft, FootRight  --> Imp
    column[27] = calculate_distance(points[14],points[18])   ## AnkleLeft, AnkleRight --> Imp
    column[28] = calculate_distance(points[15],points[21])   ## FootLeft, HandTipLeft --> Imp
    column[29] = calculate_distance(points[19],points[23])   ## FootRight, HandTipRight -->Imp
    column[30] = calculate_distance(points[7],points[11])    ## HandLeft, HandRight
    column[31] = calculate_distance(points[5],points[9])     ## ElbowLeft, ElbowRight
    column[32] = calculate_distance(points[22],points[13])   ## ThumbLeft, KneeLeft --> Imp
    column[33] = calculate_distance(points[24],points[17])   ## ThumbRight, KneeRight --> Imp
    column[34] = calculate_distance(points[7],points[12])    ## HandLeft, HipLeft
    column[35] = calculate_distance(points[11],points[16])   ## HandRight, HipRight
    column[36] = calculate_distance(points[9],points[16])    ## ElbowRight, HipRight
    column[37] = calculate_distance(points[5],points[12])    ## ElbowLeft, HipLeft

    column[38] = calculate_distance(points[3],points[4])     ## Head, ShoulderLeft
    column[39] = calculate_distance(points[3],points[8])     ## Head, ShoulderRight


    column[40] = triangle_area(points[0],points[15],points[19])   #SpineBase, FootLeft, FootRight -->Imp
    column[41] = triangle_area(points[0],points[14],points[18])   #SpineBase, AnkleLeft, AnkleRight
    column[42] = triangle_area(points[0],points[13],points[17])   #SpineBase, KneeLeft, KneeRight
    column[43] = triangle_area(points[20],points[0],points[23])   #SpineShoulder, SpineBase, HandTipRight --> Imp
    column[44] = triangle_area(points[20],points[0],points[21])   #SpineShoulder, SpineBase, HandTipLeft --> Imp
    column[45] = triangle_area(points[15],points[21],points[20])  #FootLeft, HandTipLeft, SpineShoulder --> Imp
    column[46] = triangle_area(points[19],points[23],points[20])  #FootRight, HandTipRight, SpineShoulder --> Imp
    column[47] = triangle_area(points[15],points[12],points[19])  #FootLeft, HipLeft, FootRight
    column[48] = triangle_area(points[19],points[16],points[15])  #FootRight, HipRight, FootLeft
    column[49] = triangle_area(points[3],points[21],points[12])   #Head, HandTipLeft, HipLeft
    column[50] = triangle_area(points[3],points[23],points[16])   #Head, HandTipRight, HipRight
    column[51] = triangle_area(points[3],points[23],points[19])   #Head, HandTipRight, FootRight --> Imp
    column[52] = triangle_area(points[3],points[21],points[15])   #Head, HandTipLeft, FootLeft --> Imp
    column[53] = triangle_area(points[5],points[21],points[12])   #ElbowLeft, HandTipLeft, HipLeft
    column[54] = triangle_area(points[9],points[23],points[16])   #ElbowRight, HandTipRight, HipRight
    column[55] = triangle_area(points[20],points[5],points[1])    #SpineShoulder, ElbowLeft, SpineMid
    column[56] = triangle_area(points[20],points[9],points[1])    #SpineShoulder, ElbowRight, SpineMid
    
    column[57], column[58],column[59]= calculate_centroid(points[0],points[15],points[19])
    column[60], column[61],column[62] = calculate_centroid(points[0],points[14],points[18])
    column[63], column[64],column[65] = calculate_centroid(points[0],points[13],points[17])
    column[66], column[67],column[68]= calculate_centroid(points[20],points[0],points[23])
    column[69], column[70],column[71]= calculate_centroid(points[20],points[0],points[21])
    column[72], column[73],column[74]= calculate_centroid(points[19],points[23],points[20])
    column[75], column[76],column[77] = calculate_centroid(points[15],points[21],points[20])
    column[78], column[79],column[80]= calculate_centroid(points[19],points[16],points[15])
    column[81], column[82],column[83] = calculate_centroid(points[3],points[21],points[12])
    column[84], column[85],column[86] = calculate_centroid(points[3],points[23],points[16])
    column[87], column[88],column[89] = calculate_centroid(points[3],points[23],points[19])
    column[90], column[91],column[92] = calculate_centroid(points[3],points[21],points[15])
    column[93], column[94],column[95] = calculate_centroid(points[5],points[21],points[12])
    column[96], column[97],column[98] = calculate_centroid(points[9],points[23],points[16])
    column[99], column[100],column[101] = calculate_centroid(points[20],points[5],points[1])
    column[102], column[103],column[104] = calculate_centroid(points[20],points[9],points[1])
    column[105], column[106],column[107] = calculate_centroid(points[15],points[12],points[19])
    
    column.__delitem__(1)

    return pd.DataFrame(column)


In [3]:
df = pd.read_csv(r"D:\Research_Project\My_project_22\input\raw\Both Hands front UP Left Leg Front UP\aatish_both_hand_frontup_left_leg_frontup_trial1_interpolated.csv",header=None)
x=df.iloc[:,1:]

x.shape


(37, 75)

In [4]:
x = extract_distance_features(df)
x.shape

{0: 0, 1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0, 8: 0, 9: 0, 10: 0, 11: 0, 12: 0, 13: 0, 14: 0, 15: 0, 16: 0, 17: 0, 18: 0, 19: 0, 20: 0, 21: 0, 22: 0, 23: 0, 24: 0}


(37, 107)

In [5]:
def extract_angle_features(df):
    """Returns a DataFrame with columns as the angles extracted from 3 joints."""
    SpineBase = df.iloc[:,0:3].values       #0
    SpineMid = df.iloc[:,3:6].values        #1
    Neck = df.iloc[:,6:9].values            #2
    Head = df.iloc[:,9:12].values           #3
    ShoulderLeft = df.iloc[:,12:15].values  #4
    ElbowLeft = df.iloc[:,15:18].values     #5
    WristLeft = df.iloc[:,18:21].values     #6
    HandLeft = df.iloc[:,21:24].values      #7
    ShoulderRight = df.iloc[:,24:27].values #8
    ElbowRight = df.iloc[:,27:30].values    #9
    WristRight = df.iloc[:,30:33].values    #10
    HandRight = df.iloc[:,33:36].values     #11
    HipLeft = df.iloc[:,36:39].values       #12
    KneeLeft = df.iloc[:,39:42].values      #13
    AnkleLeft = df.iloc[:,42:45].values     #14
    FootLeft = df.iloc[:,45:48].values      #15
    HipRight = df.iloc[:,48:51].values      #16
    KneeRight = df.iloc[:,51:54].values     #17
    AnkleRight = df.iloc[:,54:57].values    #18
    FootRight = df.iloc[:,57:60].values     #19
    SpineShoulder = df.iloc[:,60:63].values #20
    HandTipLeft = df.iloc[:,63:66].values   #21
    ThumbLeft = df.iloc[:,66:69].values     #22
    HandTipRight = df.iloc[:,69:72].values  #23
    ThumbRight = df.iloc[:,72:75].values    #24
    
    def calculate_angle(a1,a2,a3):
        v1 = a1 - a2
        v2 = a3 - a2
        angle = []

        #### Angle Calculation Formula 1
        # for i in range(df.shape[0]):
        #     unit_vector1 = v1[i] / np.linalg.norm(v1[i])
        #     unit_vector2 = v2[i] / np.linalg.norm(v2[i])

        #     dot_product = np.dot(unit_vector1, unit_vector2)

        #     angle.append(np.arccos(dot_product)) #angle in radian


        #### Angle Calculation Formula 2
        for i in range(df.shape[0]):
            res = (np.arctan(np.linalg.norm(np.cross(v1[i],v2[i])) / np.dot(v1[i],v2[i])) / np.pi) * 180
            angle.append(res)

        return angle

    angle_1 = calculate_angle(ShoulderLeft,ElbowLeft,WristLeft)
    angle_2 = calculate_angle(ShoulderRight,ElbowRight,WristRight)
    angle_3 = calculate_angle(Head,ShoulderLeft,HandLeft)
    angle_4 = calculate_angle(Head,ShoulderRight,HandRight)
    angle_5 = calculate_angle(SpineShoulder,ElbowLeft,ThumbLeft)
    angle_6 = calculate_angle(SpineShoulder,ElbowRight,ThumbRight)
    angle_7 = calculate_angle(SpineBase,KneeLeft,FootLeft)
    angle_8 = calculate_angle(SpineBase,KneeRight,FootRight)
    angle_9 = calculate_angle(SpineMid,HipLeft,AnkleLeft)
    angle_10 = calculate_angle(SpineMid,HipRight,AnkleRight)
    angle_11 = calculate_angle(ElbowLeft,ShoulderLeft,SpineShoulder)
    angle_12 = calculate_angle(ElbowRight,ShoulderRight,SpineShoulder)
    angle_13 = calculate_angle(ElbowRight,ShoulderRight,SpineMid)
    angle_14 = calculate_angle(ElbowLeft,ShoulderLeft,SpineMid)
    angle_15 = calculate_angle(Head,ShoulderLeft,ElbowLeft)
    angle_16 = calculate_angle(Head,ShoulderRight,ElbowRight)
    angle_17 = calculate_angle(ShoulderLeft,HipLeft,KneeLeft)
    angle_18 = calculate_angle(ShoulderRight,HipRight,KneeRight)
    angle_19 = calculate_angle(SpineBase,HipRight,KneeRight)
    angle_20 = calculate_angle(SpineBase,HipLeft,KneeLeft)
    angle_21 = calculate_angle(Neck,ShoulderLeft,ElbowLeft)
    angle_22 = calculate_angle(Neck,ShoulderRight,ElbowRight)
    angle_23 = calculate_angle(KneeLeft,AnkleLeft,FootLeft)
    angle_24 = calculate_angle(KneeRight,AnkleRight,FootRight)
    
    
    angle_df = pd.DataFrame(data=[angle_1,angle_2,angle_3,angle_4,angle_5,angle_6,angle_7,angle_8,angle_9,angle_10,angle_11,angle_12,angle_13,angle_14,angle_15,angle_16,angle_17,angle_18,angle_19,angle_20,angle_21,angle_22,angle_23,angle_24]).T
    
    return angle_df

In [6]:
x = extract_angle_features(df)
x

,0,1,2,3,4,5,6,7,8,9,...,14,15,16,17,18,19,20,21,22,23
0,-7.368696,-7.913459,-28.869848,-27.530457,-41.816535,-36.414347,-60.491758,-62.940441,-33.265964,-18.571875,...,-27.054901,-23.413283,-19.393515,-31.476200,76.513647,-79.136398,-46.733883,-52.504020,-38.546464,-47.380452
1,-7.852081,-4.563777,-31.135832,-24.727450,-43.004244,-25.780686,-61.187794,-64.423577,-29.106071,-18.918380,...,-27.620561,-21.878494,-16.105044,-29.634623,77.029206,-82.222986,-49.104136,-50.408647,-34.248397,-40.332393
2,-8.128025,-4.173059,-31.718334,-24.124485,-43.679404,-22.794142,-61.302072,-64.852305,-27.525934,-18.990114,...,-27.818462,-21.725726,-15.302338,-28.859748,77.270613,-83.343503,-49.891088,-49.790573,-32.506853,-37.749620
3,-8.232526,-4.224994,-31.555832,-24.329007,-43.837892,-23.074060,-61.149375,-64.759275,-27.595439,-18.949195,...,-27.780405,-21.854199,-15.421030,-28.726214,77.355057,-83.281524,-49.922007,-49.817784,-32.356631,-37.305004
4,-8.346082,-4.413132,-31.474581,-24.406456,-43.901374,-23.155192,-61.020715,-64.574451,-28.358005,-18.917447,...,-27.673842,-22.169455,-15.952316,-28.843829,77.395554,-82.734897,-49.888669,-50.111472,-32.414396,-37.370623
5,-8.829942,-5.354528,-31.295274,-25.011818,-43.623339,-28.922462,-60.918090,-64.418552,-29.730056,-18.850849,...,-27.694223,-23.383972,-17.015954,-29.323546,77.499301,-81.750322,-49.940075,-51.232461,-32.476370,-37.462092
6,-10.059181,-6.367682,-31.190299,-25.586472,-43.327242,-26.960652,-60.759504,-64.220900,-31.055594,-18.813197,...,-27.751174,-25.119440,-18.094938,-30.073149,77.503812,-80.835489,-50.171641,-52.850791,-32.519346,-37.476132
7,-10.962734,-9.172848,-30.824049,-26.367707,-42.120377,-27.765868,-60.665707,-64.061042,-33.392045,-18.759955,...,-27.871214,-28.464883,-20.161568,-31.233279,77.585277,-79.205205,-50.378189,-55.856283,-32.800199,-37.507823
8,-13.084024,-19.948875,-30.156805,-28.839581,-41.019502,-26.151853,-60.675799,-64.148541,-42.666026,-18.529590,...,-27.864847,-39.993252,-29.269991,-34.697989,77.873304,-73.114313,-50.625917,-65.863887,-33.391875,-37.646397
9,-17.423207,-37.391330,-30.045580,-35.829770,-42.180732,-30.265281,-60.911019,-64.308850,-53.799017,-18.094051,...,-27.809216,-59.163235,-41.466133,-39.590680,78.421072,-65.860184,-50.914011,-82.704071,-33.963540,-38.194973


#### CREATING EXTRACTED FEATURES CSV

In [7]:
INPUT_PATH = r'D:\Research_Project\My_project_22\input\raw'
ANGLES_PATH = r'D:\Research_Project\My_project_22\FEATURES_EXTRACTED\ANGLES'
DISTANCES_PATH = r"D:\Research_Project\My_project_22\FEATURES_EXTRACTED\DISTANCES"

In [8]:
raw_input_files = glob(os.path.join(INPUT_PATH,"*","*_interpolated.csv"))
print(raw_input_files)

['D:\\Research_Project\\My_project_22\\input\\raw\\Both Hands front UP Left Leg Front UP\\aatish_both_hand_frontup_left_leg_frontup_trial1_interpolated.csv', 'D:\\Research_Project\\My_project_22\\input\\raw\\Both Hands front UP Left Leg Front UP\\aatish_both_hand_frontup_left_leg_frontup_trial2_interpolated.csv', 'D:\\Research_Project\\My_project_22\\input\\raw\\Both Hands front UP Left Leg Front UP\\arnab_both_hand_frontup_left_leg_frontup_trial1_interpolated.csv', 'D:\\Research_Project\\My_project_22\\input\\raw\\Both Hands front UP Left Leg Front UP\\arnab_both_hand_frontup_left_leg_frontup_trial2_interpolated.csv', 'D:\\Research_Project\\My_project_22\\input\\raw\\Both Hands front UP Left Leg Front UP\\Harsh_both_hand_frontup_left_leg_frontup_trial1_interpolated.csv', 'D:\\Research_Project\\My_project_22\\input\\raw\\Both Hands front UP Left Leg Front UP\\Harsh_both_hand_frontup_left_leg_frontup_trial2_interpolated.csv', 'D:\\Research_Project\\My_project_22\\input\\raw\\Both Hands 

In [9]:
def save_csv_dir(x,path):
    file_name_splits = os.path.basename(x).split('_')
    trial_string_idx = [i for i, elem in enumerate(file_name_splits) if re.search('trial*', elem)][0]
    gesture = "_".join(file_name_splits[1:trial_string_idx])

    save_dir = os.path.join(path,gesture)
    if not os.path.exists(save_dir):
        os.makedirs(save_dir)

    return save_dir

def extract_participant_pose_name(x):
    x = x.split("\\")[-1]
    return x

In [10]:
for i in range(len(raw_input_files)):
    data =pd.read_csv(raw_input_files[i],header=None)
    data = data.iloc[:,1:]
    folder_name = extract_participant_pose_name(raw_input_files[i])
    # print(folder_name)
    distance_df = extract_distance_features(data)
    angle_df = extract_angle_features(data)
    angles_save_dir = save_csv_dir(raw_input_files[i],ANGLES_PATH)
    distances_save_dir = save_csv_dir(raw_input_files[i],DISTANCES_PATH)
    angles_save_path = os.path.join(angles_save_dir,folder_name)
    distances_save_path = os.path.join(distances_save_dir,folder_name)
    distance_df.to_csv(distances_save_path,index=None,header=None)
    angle_df.to_csv(angles_save_path,index=None,header=None)
        

{0: 0, 1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0, 8: 0, 9: 0, 10: 0, 11: 0, 12: 0, 13: 0, 14: 0, 15: 0, 16: 0, 17: 0, 18: 0, 19: 0, 20: 0, 21: 0, 22: 0, 23: 0, 24: 0}
{0: 0, 1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0, 8: 0, 9: 0, 10: 0, 11: 0, 12: 0, 13: 0, 14: 0, 15: 0, 16: 0, 17: 0, 18: 0, 19: 0, 20: 0, 21: 0, 22: 0, 23: 0, 24: 0}
{0: 0, 1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0, 8: 0, 9: 0, 10: 0, 11: 0, 12: 0, 13: 0, 14: 0, 15: 0, 16: 0, 17: 0, 18: 0, 19: 0, 20: 0, 21: 0, 22: 0, 23: 0, 24: 0}
{0: 0, 1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0, 8: 0, 9: 0, 10: 0, 11: 0, 12: 0, 13: 0, 14: 0, 15: 0, 16: 0, 17: 0, 18: 0, 19: 0, 20: 0, 21: 0, 22: 0, 23: 0, 24: 0}
{0: 0, 1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0, 8: 0, 9: 0, 10: 0, 11: 0, 12: 0, 13: 0, 14: 0, 15: 0, 16: 0, 17: 0, 18: 0, 19: 0, 20: 0, 21: 0, 22: 0, 23: 0, 24: 0}
{0: 0, 1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0, 8: 0, 9: 0, 10: 0, 11: 0, 12: 0, 13: 0, 14: 0, 15: 0, 16: 0, 17: 0, 18: 0, 19: 0, 20: 0, 21: 0, 22: 0, 23: 0, 24: 0}
{0: 

In [101]:
from glob import glob
import pandas as pd
from csv import writer
from csv import reader
# def add_column_in_csv(input_file, output_file, transform_row):
#     """ Append a column in existing csv using csv.reader / csv.writer classes"""
#     # Open the input_file in read mode and output_file in write mode
#     with open(input_file, 'r') as read_obj, \
#             open(output_file, 'w', newline='') as write_obj:
#         # Create a csv.reader object from the input file object
#         csv_reader = reader(read_obj)
#         # Create a csv.writer object from the output file object
#         csv_writer = writer(write_obj)
#         # Read each row of the input csv file as list
#         for row in csv_reader:
#             # Pass the list / row in the transform function to add column text for this row
#             transform_row(row, csv_reader.line_num)
#             # Write the updated row / list to the output file
#             csv_writer.writerow(row)
# raw_angle_files_1 = glob(os.path.join(config.INPUT_DISTANCES, "*", "*.csv"))
# all_filenames = [i for i in raw_angle_files_1]
# df=pd.read_csv(all_filenames[0],header=None)
# df.head(10)
# add_column_in_csv(all_filenames[0], all_filenames[0], lambda row, line_num: row.append(2))
# for x in all_filenames:
#     df=pd.read_csv(x)
#     print(df.shape)
# all_filename=[all_filenames[0],all_filenames[1]]
# df = pd.concat(map(pd.read_csv, all_filenames), axis=1)
# # print(df)
# df.shape
# raw_angle_files_1 = glob(os.path.join(config.INPUT_PREPROCESSED, "*", "*.csv"))
# # print(raw_angle_files_1)
# all_filenames = [i for i in raw_angle_files_1]
# df=pd.read_csv(all_filenames[0],header=None)
# df[90]=0
# df.to_csv(all_filenames[0], index=False)
# df=pd.read_csv(all_filenames[1],header=None)
# df[90]=0
# df.to_csv(all_filenames[1], index=False)
# all_filename=[all_filenames[0],all_filenames[1]]
# df = pd.concat(map(pd.read_csv, all_filenames),ignore_index=True)
# df.shape
# print(len(all_filenames))
# for i in range(len(all_filenames)):
#     if i==0 or i==1:
#         continue
#     df=pd.read_csv(all_filenames[0],header=None)
#     df[90]=0
dir_list = os.listdir("D:\Research_Project\My_project_22\FEATURES_EXTRACTED\DISTANCES") 
count=0
# csv_files= glob(os.path.join(config.INPUT_DISTANCES, dir_list[0], "*.csv"))
# all_csv_files = [j for j in csv_files ]
# for i in range(len(all_csv_files)):
#     df=pd.read_csv(all_csv_files[i],header=None)
#     df[90]=0
#     df.to_csv(all_filenames[i],index=False)

# adding labels
for i in range(len(dir_list)):
    csv_files= glob(os.path.join("D:\Research_Project\My_project_22\FEATURES_EXTRACTED\DISTANCES", dir_list[i], "*.csv"))
    all_csv_files = [j for j in csv_files ]

    for j in range(len(all_csv_files)):
        df=pd.read_csv(all_csv_files[j],header=None)
        print(df.shape)
        df['107']=i
        df.to_csv(all_csv_files[j],index=False) 
       
   
#merging
# for i in range(len(dir_list)):
#     csv_files= glob(os.path.join(config.INPUT_PREPROCESSED, dir_list[i], "*.csv"))
#     all_csv_files = [j for j in csv_files ]
     
#     for j in range(len(all_csv_files)):
#         df=pd.read_csv(all_csv_files[j],header=None)
#         df[90]=i
#         df.to_csv(all_csv_files[j],index=False) 

#train
# from sklearn.model_selection import train_test_split
# df = pd.concat(map(pd.read_csv, all_filenames),ignore_index=True)
# # print(df.shape)
# df = df.sample(frac = 1)
# # df.iloc[1:10,:]
# # print(df['26'])
# # df = df.sample(frac = 1)
# data=df
# # # df.head(10)
# # # print(df[0])
# target=data['26']
# data=data.drop(['26'],axis=1)
# # # print(target)
# # # target.shape
# x_train,x_test,y_train,y_test= train_test_split(data, target, test_size = 0.2)
# x_train.head(10)
# y_train.head(10)
# x_train=x_train.to_numpy()
# x_test=x_test.to_numpy()
# y_train=y_train.to_numpy()
# y_test=y_test.to_numpy()
# print(type(x_train))
# # print(x_train.head(10))
# print(y_train)
# print(x_test.shape)
# print(y_train.shape)
# print(y_train)
# print(x_train.head(10))
# x_train.head(10)
# y_train.head(10)
# df.iloc[10:20,:]

(37, 107)
(60, 107)
(82, 107)
(65, 107)
(65, 107)
(75, 107)
(62, 107)
(50, 107)
(77, 107)
(67, 107)
(77, 107)
(87, 107)
(57, 107)
(55, 107)
(77, 107)
(70, 107)
(120, 107)
(107, 107)
(50, 107)
(37, 107)
(37, 107)
(57, 107)
(60, 107)
(65, 107)
(75, 107)
(77, 107)
(57, 107)
(55, 107)
(87, 107)
(90, 107)
(62, 107)
(42, 107)
(67, 107)
(60, 107)
(60, 107)
(55, 107)
(90, 107)
(90, 107)
(72, 107)
(75, 107)
(42, 107)
(52, 107)
(82, 107)
(82, 107)
(70, 107)
(70, 107)
(60, 107)
(65, 107)
(92, 107)
(102, 107)
(70, 107)
(82, 107)
(65, 107)
(72, 107)
(70, 107)
(67, 107)
(100, 107)
(107, 107)
(57, 107)
(57, 107)
(60, 107)
(62, 107)
(72, 107)
(70, 107)
(72, 107)
(75, 107)
(47, 107)
(70, 107)
(100, 107)
(87, 107)
(55, 107)
(60, 107)
(60, 107)
(62, 107)
(65, 107)
(70, 107)
(115, 107)
(92, 107)
(57, 107)
(65, 107)
(52, 107)
(60, 107)
(55, 107)
(60, 107)
(77, 107)
(75, 107)
(60, 107)
(62, 107)
(80, 107)
(92, 107)
(72, 107)
(92, 107)
(60, 107)
(45, 107)
(67, 107)
(67, 107)
(100, 107)
(90, 107)
(65, 107)
(7

In [78]:
from glob import glob
import pandas as pd
from csv import writer
from csv import reader
# def add_column_in_csv(input_file, output_file, transform_row):
#     """ Append a column in existing csv using csv.reader / csv.writer classes"""
#     # Open the input_file in read mode and output_file in write mode
#     with open(input_file, 'r') as read_obj, \
#             open(output_file, 'w', newline='') as write_obj:
#         # Create a csv.reader object from the input file object
#         csv_reader = reader(read_obj)
#         # Create a csv.writer object from the output file object
#         csv_writer = writer(write_obj)
#         # Read each row of the input csv file as list
#         for row in csv_reader:
#             # Pass the list / row in the transform function to add column text for this row
#             transform_row(row, csv_reader.line_num)
#             # Write the updated row / list to the output file
#             csv_writer.writerow(row)
# raw_angle_files_1 = glob(os.path.join(config.INPUT_DISTANCES, "*", "*.csv"))
# all_filenames = [i for i in raw_angle_files_1]
# df=pd.read_csv(all_filenames[0],header=None)
# df.head(10)
# add_column_in_csv(all_filenames[0], all_filenames[0], lambda row, line_num: row.append(2))
# for x in all_filenames:
#     df=pd.read_csv(x)
#     print(df.shape)
# all_filename=[all_filenames[0],all_filenames[1]]
# df = pd.concat(map(pd.read_csv, all_filenames), axis=1)
# # print(df)
# df.shape
# raw_angle_files_1 = glob(os.path.join(config.INPUT_PREPROCESSED, "*", "*.csv"))
# # print(raw_angle_files_1)
# all_filenames = [i for i in raw_angle_files_1]
# df=pd.read_csv(all_filenames[0],header=None)
# df[90]=0
# df.to_csv(all_filenames[0], index=False)
# df=pd.read_csv(all_filenames[1],header=None)
# df[90]=0
# df.to_csv(all_filenames[1], index=False)
# all_filename=[all_filenames[0],all_filenames[1]]
# df = pd.concat(map(pd.read_csv, all_filenames),ignore_index=True)
# df.shape
# print(len(all_filenames))
# for i in range(len(all_filenames)):
#     if i==0 or i==1:
#         continue
#     df=pd.read_csv(all_filenames[0],header=None)
#     df[90]=0
dir_list = os.listdir(r"D:\Research_Project\My_project_22\input\raw") 
count=0
# print(dir_list)
# csv_files= glob(os.path.join(config.INPUT_DISTANCES, dir_list[0], "*.csv"))
# all_csv_files = [j for j in csv_files ]
# for i in range(len(all_csv_files)):
#     df=pd.read_csv(all_csv_files[i],header=None)
#     df[90]=0
#     df.to_csv(all_filenames[i],index=False)

# adding labels
INPUT_PATH = r'D:\Research_Project\My_project_22\input\raw'
for i in range(len(dir_list)):
    # print(dir_list[i])
  
    csv_files= glob(os.path.join(INPUT_PATH, dir_list[i],"ANGLES","*_interpolated_ANGLES.csv"))
    # print(csv_files)
    all_csv_files = [j for j in csv_files ]

    for j in range(len(all_csv_files)):
        df=pd.read_csv(all_csv_files[j],header=None)
        print(df.shape)
        df[17]=i
        df.to_csv(all_csv_files[j],index=False) 
       
   
#merging
# for i in range(len(dir_list)):
#     csv_files= glob(os.path.join(config.INPUT_PREPROCESSED, dir_list[i], "*.csv"))
#     all_csv_files = [j for j in csv_files ]
     
#     for j in range(len(all_csv_files)):
#         df=pd.read_csv(all_csv_files[j],header=None)
#         df[90]=i
#         df.to_csv(all_csv_files[j],index=False) 

#train
# from sklearn.model_selection import train_test_split
# df = pd.concat(map(pd.read_csv, all_filenames),ignore_index=True)
# # print(df.shape)
# df = df.sample(frac = 1)
# # df.iloc[1:10,:]
# # print(df['26'])
# # df = df.sample(frac = 1)
# data=df
# # # df.head(10)
# # # print(df[0])
# target=data['26']
# data=data.drop(['26'],axis=1)
# # # print(target)
# # # target.shape
# x_train,x_test,y_train,y_test= train_test_split(data, target, test_size = 0.2)
# x_train.head(10)
# y_train.head(10)
# x_train=x_train.to_numpy()
# x_test=x_test.to_numpy()
# y_train=y_train.to_numpy()
# y_test=y_test.to_numpy()
# print(type(x_train))
# # print(x_train.head(10))
# print(y_train)
# print(x_test.shape)
# print(y_train.shape)
# print(y_train)
# print(x_train.head(10))
# x_train.head(10)
# y_train.head(10)
# df.iloc[10:20,:]

(37, 17)
(60, 17)
(82, 17)
(65, 17)
(65, 17)
(75, 17)
(62, 17)
(50, 17)
(77, 17)
(67, 17)
(77, 17)
(87, 17)
(57, 17)
(55, 17)
(77, 17)
(70, 17)
(120, 17)
(50, 17)
(37, 17)
(37, 17)
(57, 17)
(60, 17)
(65, 17)
(75, 17)
(77, 17)
(57, 17)
(55, 17)
(87, 17)
(90, 17)
(62, 17)
(42, 17)
(67, 17)
(60, 17)
(60, 17)
(55, 17)
(90, 17)
(90, 17)
(72, 17)
(75, 17)
(42, 17)
(52, 17)
(82, 17)
(82, 17)
(70, 17)
(70, 17)
(60, 17)
(65, 17)
(92, 17)
(102, 17)
(70, 17)
(82, 17)
(65, 17)
(72, 17)
(70, 17)
(67, 17)
(100, 17)
(107, 17)
(57, 17)
(57, 17)
(60, 17)
(62, 17)
(72, 17)
(70, 17)
(72, 17)
(75, 17)
(47, 17)
(70, 17)
(100, 17)
(87, 17)
(55, 17)
(60, 17)
(60, 17)
(62, 17)
(65, 17)
(70, 17)
(115, 17)
(92, 17)
(57, 17)
(65, 17)
(52, 17)
(60, 17)
(55, 17)
(60, 17)
(77, 17)
(75, 17)
(60, 17)
(62, 17)
(80, 17)
(92, 17)
(72, 17)
(92, 17)
(60, 17)
(45, 17)
(67, 17)
(67, 17)
(90, 17)
(100, 17)
(65, 17)
(70, 17)
(45, 17)
(50, 17)
(40, 17)
(52, 17)
(65, 17)
(90, 17)
(80, 17)
(80, 17)
(65, 17)
(62, 17)
(85, 17)
(82

In [24]:
import tensorflow as tf
from tensorflow.keras.datasets import imdb
from tensorflow.keras.layers import Embedding, Dense, LSTM
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Model configuration
additional_metrics = ['accuracy']
batch_size = 128
embedding_output_dims = 15
loss_function = BinaryCrossentropy()
max_sequence_length = 300
num_distinct_words = 5000
number_of_epochs = 5
optimizer = Adam()
validation_split = 0.20
verbosity_mode = 1

# Disable eager execution
tf.compat.v1.disable_eager_execution()

# Load dataset
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=num_distinct_words)
print(type(x_train))
print(type(y_train))
print(type(x_test))
print(type(y_test))
print(x_train.shape)
print(x_test.shape)
print(x_train.head(10))

# # Pad all sequences
# padded_inputs = pad_sequences(x_train, maxlen=max_sequence_length, value = 0.0) # 0.0 because it corresponds with <PAD>
# padded_inputs_test = pad_sequences(x_test, maxlen=max_sequence_length, value = 0.0) # 0.0 because it corresponds with <PAD>

# # Define the Keras model
# model = Sequential()
# model.add(Embedding(num_distinct_words, embedding_output_dims, input_length=max_sequence_length))
# model.add(LSTM(10))
# model.add(Dense(1, activation='sigmoid'))

# # Compile the model
# model.compile(optimizer=optimizer, loss=loss_function, metrics=additional_metrics)

# # Give a summary
# model.summary()

# # Train the model
# history = model.fit(padded_inputs, y_train, batch_size=batch_size, epochs=number_of_epochs, verbose=verbosity_mode, validation_split=validation_split)

# # Test the model after training
# test_results = model.evaluate(padded_inputs_test, y_test, verbose=False)
# print(f'Test results - Loss: {test_results[0]} - Accuracy: {100*test_results[1]}%')

<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
(25000,)
(25000,)


AttributeError: 'numpy.ndarray' object has no attribute 'head'

In [21]:

import config
raw_angle_files_1 = glob(os.path.join(config.INPUT_DISTANCES, "*", "*.csv"))
file_name_splits = os.path.basename(raw_angle_files_1[0]).split('_')
print(file_name_splits)


ModuleNotFoundError: No module named 'config'